In [ ]:
%pip install anthropic
%pip install python-dotenv
%pip install rich


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
from dotenv import load_dotenv
from anthropic import Anthropic

# Load env variables
load_dotenv()

# Create Anthropic client
client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

In [9]:
MODEL = "claude-3-7-sonnet-latest"

In [17]:
response = client.messages.create(
    model=MODEL,
    max_tokens=1024,
    system="Be concise.",
    messages=[
        {"role": "user", "content": "Give me a short hello world response"}
    ]
)

In [ ]:
from rich import print as rprint

In [23]:
rprint(response)

Message(
    id='msg_01XvHJLMKfmssN7gbkep5gua',
    content=[TextBlock(citations=None, text='Hello, World!', type='text')],
    model='claude-3-7-sonnet-latest',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=Usage(
        cache_creation_input_tokens=0,
        cache_read_input_tokens=0,
        input_tokens=18,
        output_tokens=7,
        server_tool_use=None
    )
)

In [24]:
response_2 = client.messages.create(
    model=MODEL,
    max_tokens=1024,
    system="Be concise.",
    messages=[
        {"role": "user", "content": "Give me a short good bye response"}
    ]
)

In [25]:
rprint(response_2)

Message(
    id='msg_019H11jPshKLLzTiJXh3sGNv',
    content=[
        TextBlock(citations=None, text='Farewell! Thanks for chatting with me today. Take care!', type='text')
    ],
    model='claude-3-7-sonnet-latest',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=Usage(
        cache_creation_input_tokens=0,
        cache_read_input_tokens=0,
        input_tokens=18,
        output_tokens=18,
        server_tool_use=None
    )
)

In [35]:
TOOLS = [
    {
        "name": "get_weather",
        "description": "Get the current weather in a given location",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                }
            },
            "required": ["location"],
        },
    }
]

In [38]:
response_3 = client.messages.create(
  model=MODEL,
  max_tokens=1024,
  system="You are a helpful assistant that can answer questions and help with tasks.",
  tools=TOOLS,
  tool_choice={"type": "any"},
  messages=[
    {
      "role": "user",
      "content": "What is the weather in Tokyo in the morning?"
    }
  ]
)

In [39]:
rprint(response_3)

Message(
    id='msg_01BguexfM7dnK2M8CfqD3c72',
    content=[
        ToolUseBlock(
            id='toolu_01GzoTpCaZ1z8KeK4CsGDPDF',
            input={'location': 'Tokyo, Japan'},
            name='get_weather',
            type='tool_use'
        )
    ],
    model='claude-3-7-sonnet-latest',
    role='assistant',
    stop_reason='tool_use',
    stop_sequence=None,
    type='message',
    usage=Usage(
        cache_creation_input_tokens=0,
        cache_read_input_tokens=0,
        input_tokens=420,
        output_tokens=40,
        server_tool_use=None
    )
)

In [43]:
rprint(response_3.content)

[
    ToolUseBlock(
        id='toolu_01GzoTpCaZ1z8KeK4CsGDPDF',
        input={'location': 'Tokyo, Japan'},
        name='get_weather',
        type='tool_use'
    )
]

In [47]:
response_4 = client.messages.create(
    model=MODEL,
    max_tokens=1024,
    system="You are a helpful assistant that can answer questions and help with tasks.",
    tools=TOOLS,
    tool_choice={"type": "auto"},
    messages=[
        {"role": "assistant", "content": response_3.content},
        {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": response_3.content[0].id,
                    "content": "sunny",
                }
            ],
        },
    ],
)

In [48]:
rprint(response_4)

Message(
    id='msg_01GUXzU5YKuDYoGw3ocizLKS',
    content=[
        TextBlock(
            citations=None,
            text="The current weather in Tokyo, Japan is sunny! It's a beautiful day there. Is there anything else 
you'd like to know about the weather in Tokyo or any other location?",
            type='text'
        )
    ],
    model='claude-3-7-sonnet-latest',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=Usage(
        cache_creation_input_tokens=0,
        cache_read_input_tokens=0,
        input_tokens=486,
        output_tokens=40,
        server_tool_use=None
    )
)